In [6]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, GenerationConfig
from dotenv import load_dotenv
from huggingface_hub import login 
from deploy_utils import fit_into_model, gen_prompt, get_starter_prompt, check_gpu, get_model_tokenizer

# Load the environment file
load_dotenv("token.env")

# Retrieve the API token
api_token = os.getenv("api_token")
print(api_token)
# Log in to the Hugging Face Hub
login(token = api_token)

# Check if CUDA is available
check_gpu()

None


Number of GPUs available: 1
GPU 0: NVIDIA GeForce RTX 3060


In [7]:
# Define the model name
model, tokenizer = get_model_tokenizer("google/gemma-7b-it")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
import sys
sys.path.insert(0, '..')

from retrival.helper_functions import get_embeddings, create_vector_db, similarity_search

In [4]:
docs, embeddings = get_embeddings()
db = create_vector_db(docs, embeddings, 'instructions')

Created a chunk of size 214, which is longer than the specified 100
Created a chunk of size 430, which is longer than the specified 100
Created a chunk of size 311, which is longer than the specified 100
Created a chunk of size 924, which is longer than the specified 100
Created a chunk of size 328, which is longer than the specified 100


In [16]:
query = "start the game"
query_docs = db.similarity_search(query, k = 2)
print(query_docs[1].page_content)

[General Gameplay/Start Game]
- Provide occasional hints, tips, or prompts to guide the player's decision-making process.
- Encourage creativity and exploration while maintaining a sense of challenge and adventure.


In [9]:
prompt = get_starter_prompt(tokenizer)
print(prompt)

<bos><start_of_turn>user
Play a game with me where i am in an enchanted forest full of beasts and loots and make it imaginative.
You be the narrator and i will be the player, play a dialog game with me. Here are some set of instructions to help you out:
[General Gameplay/Start Game]
- Provide occasional hints, tips, or prompts to guide the player's decision-making process.
- Encourage creativity and exploration while maintaining a sense of challenge and adventure.

[Exploration]
- The player finds themselves in a dense, enchanted forest teeming with mystical creatures and hidden treasures.
- Encourage the player to explore the forest by describing the surroundings, potential encounters, and points of interest.
- Use vivid imagery and descriptive language to immerse the player in the forest environment.
- Provide hints or clues about nearby landmarks, valuable items, or potential dangers.

[What to do]
- Exploration: The player can move deeper into the forest to discover new areas and e

In [10]:
prompt = get_starter_prompt(tokenizer)
for i in range(0,100):
    output = fit_into_model(prompt, tokenizer, model)
    print(output)
    user_input = input('What is your next move?')
    prompt = gen_prompt(user_input,output, model)

<bos><start_of_turn>user
Play a game with me where i am in an enchanted forest full of beasts and loots and make it imaginative.
You be the narrator and i will be the player, play a dialog game with me. Here are some set of instructions to help you out:
[General Gameplay/Start Game]
- Provide occasional hints, tips, or prompts to guide the player's decision-making process.
- Encourage creativity and exploration while maintaining a sense of challenge and adventure.

[Exploration]
- The player finds themselves in a dense, enchanted forest teeming with mystical creatures and hidden treasures.
- Encourage the player to explore the forest by describing the surroundings, potential encounters, and points of interest.
- Use vivid imagery and descriptive language to immerse the player in the forest environment.
- Provide hints or clues about nearby landmarks, valuable items, or potential dangers.

[What to do]
- Exploration: The player can move deeper into the forest to discover new areas and e